In [5]:
import requests
from bs4 import BeautifulSoup
import bs4
import json
from collections import defaultdict
import pandas as pd
from datetime import datetime
import codecs
from sklearn.preprocessing import MultiLabelBinarizer
import wget
import os

In [249]:
dataframe_columns = ['type','dateCreated', 'datePublished','reviewBody','url','author_type','author_name','url','content_type','movie_name','review_type','ratingValue','tomatometer']

#fields extracted from html page
review_fields = ['@type','dateCreated', 'datePublished','reviewBody','url']
author_fields = ['@type','name','url']
content_fields = ['@type','name']
rating_fields = ['@type','ratingValue','tomatometer']

#movie features
features = ['Rated','Runtime','Genre','Director','Writer','Actor','Plot','Language','Country','Poster','Production','Response']
feature_index = [2,4,5,6,7,8,9,10,11,13,22,24]
feature_dict = dict(zip(feature_index,features))

#valid years
valid_years = [2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008]

In [224]:
mlb = MultiLabelBinarizer()

In [80]:
reader = codecs.getreader('utf-8')

In [3]:
#get page html
def get_page(page_num):
    URL = "https://www.rottentomatoes.com/source-337?page={}".format(page_num)
    page = requests.get(URL)

    soup = BeautifulSoup(page.content,"html.parser")
    return soup

In [4]:
#convert page data to pandas dataframe
def page_to_df(soup_data):
    data = []
    
    reviews = soup_data.find("script",{"type":"application/ld+json"}).contents[0] #extract unlcean review data
    reviews = reviews.strip() #remove new line characters
    reviews = json.loads(reviews) #convert to dictionary
    reviews = reviews['@graph'][1]['itemListElement'] #extract clean review data
    
    for review in reviews:
        rows = []
        for re_f in review_fields:
            rows.append(review['item'][re_f])
        for au_f in author_fields:
            rows.append(review['item']['author'][au_f])
        for co_f in content_fields:
            rows.append(review['item']['itemReviewed'][co_f])
        for ra_f in rating_fields:
            rows.append(review['item']['reviewRating'][ra_f])
        data.append(rows)
    
    return pd.DataFrame(data,columns = dataframe_columns)

In [ ]:
def get_movie_id(movie):
    res = ia.search_move(movie)
    for r in res:
        if r['title'] == Movie:
            return r['']

In [5]:
#loop through rotten tomatoes pages and get data
all_reviews = []
total_review_pages = 378
for page in range(1,total_review_pages+1):
    try:
        all_reviews.append(page_to_df(get_page(page)))
    except:
        pass

1 2 3 4 5 Error 7 8 9 10 11 12 13 Error Error Error Error Error 19 Error 21 Error Error 24 Error Error 27 28 Error Error Error Error Error Error Error Error Error 38 Error Error Error Error Error Error Error Error Error Error 49 Error Error Error Error Error Error Error Error Error Error 60 Error Error Error Error Error Error Error Error Error Error Error Error Error 74 75 Error Error 78 Error 80 81 82 83 84 85 86 87 88 89 90 91 92 93 Error 95 96 97 98 99 Error 101 102 103 104 105 106 107 108 109 110 111 Error Error Error 115 116 117 118 119 120 121 122 123 124 125 126 Error 128 129 130 131 132 133 Error 135 136 137 138 139 Error 141 142 143 144 145 Error 147 Error Error 150 151 Error 153 154 Error 156 Error 158 159 Error 161 162 163 164 165 166 Error 168 169 170 171 172 173 174 175 176 177 178 Error 180 Error 182 183 184 Error 186 187 188 189 190 191 192 193 Error 195 Error 197 Error 199 200 201 202 203 Error 205 206 Error 208 209 Error 211 212 213 214 Error 216 Error Error 219 220 22

In [267]:
def call_api(movie,year):
    url = "http://www.omdbapi.com/?apikey=69c24e49&t={}&y={}".format(movie,year)
    response = requests.get(url).json()
    return response

In [ ]:
def call_api_image(movie,year):
    url = "http://img.omdbapi.com/?apikey=69c24e49&t={}&y={}".format(movie,year)
    

In [198]:
#account for writer vs writers, actor vs actors etc.
def data_reformat(res):
    curr = []
    for f in features:
        if(f in res.keys()):
            curr.append(res[f])
        elif(f+"s" in res.keys()):
            curr.append(res[f+"s"])
        else:
            curr.append("N/A")
    return dict(zip(features,curr))

In [7]:
#compile scraped dataframes into a single frame and export as csv
total_reviews = pd.concat(all_reviews)
total_reviews.to_csv("total_reviews.csv",index = False)

In [233]:
data = pd.read_csv('total_reviews.csv')

In [254]:
data['year'] = data['dateCreated'].apply(lambda date: datetime.strptime(date,'%b %d, %Y').year)

In [270]:
#get additional features from omdb api 
extra_data = []
for n in range(len(data)):
    extra_data.append(
        call_api(
            data['movie_name'][n],
            data['year'][n]
        )
    )
    print(n,end='')

0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300301302303304305306307308309310311312313314315316317318319320321322323324325326327328329330331332333334335336337338339340341342343344345346347348349350351352353354355356357358359360361362363364365366367368369

2328232923302331233223332334233523362337233823392340234123422343234423452346234723482349235023512352235323542355235623572358235923602361236223632364236523662367236823692370237123722373237423752376237723782379238023812382238323842385238623872388238923902391239223932394239523962397239823992400240124022403240424052406240724082409241024112412241324142415241624172418241924202421242224232424242524262427242824292430243124322433243424352436243724382439244024412442244324442445244624472448244924502451245224532454245524562457245824592460246124622463246424652466246724682469247024712472247324742475247624772478247924802481248224832484248524862487248824892490249124922493249424952496249724982499250025012502250325042505250625072508250925102511251225132514251525162517251825192520252125222523252425252526252725282529253025312532253325342535253625372538253925402541254225432544254525462547254825492550255125522553255425552556255725582559256025612562256325642565256625672568256925702571257225732574257525762577

4378437943804381438243834384438543864387438843894390439143924393439443954396439743984399440044014402440344044405440644074408440944104411441244134414441544164417441844194420442144224423442444254426442744284429443044314432443344344435443644374438443944404441444244434444444544464447444844494450445144524453445444554456445744584459446044614462446344644465446644674468446944704471447244734474447544764477447844794480448144824483448444854486448744884489449044914492449344944495449644974498449945004501450245034504450545064507450845094510451145124513451445154516451745184519452045214522452345244525452645274528452945304531453245334534453545364537453845394540454145424543454445454546454745484549455045514552455345544555455645574558455945604561456245634564456545664567456845694570457145724573457445754576457745784579458045814582458345844585458645874588458945904591459245934594459545964597459845994600460146024603460446054606460746084609461046114612461346144615461646174618461946204621462246234624462546264627

6428642964306431643264336434643564366437643864396440644164426443644464456446644764486449645064516452645364546455645664576458645964606461646264636464646564666467646864696470647164726473647464756476647764786479648064816482648364846485648664876488648964906491649264936494649564966497649864996500650165026503650465056506650765086509651065116512651365146515651665176518651965206521652265236524652565266527652865296530653165326533653465356536653765386539654065416542654365446545654665476548654965506551655265536554655565566557655865596560656165626563656465656566656765686569657065716572657365746575657665776578657965806581658265836584658565866587658865896590659165926593659465956596659765986599660066016602660366046605660666076608660966106611661266136614661566166617661866196620662166226623662466256626662766286629663066316632663366346635663666376638663966406641664266436644664566466647664866496650665166526653665466556656665766586659666066616662666366646665666666676668666966706671667266736674667566766677

8477847884798480848184828483848484858486848784888489849084918492849384948495849684978498849985008501850285038504850585068507850885098510851185128513851485158516851785188519852085218522852385248525852685278528852985308531853285338534853585368537853885398540854185428543854485458546854785488549855085518552855385548555855685578558855985608561856285638564856585668567856885698570857185728573857485758576857785788579858085818582858385848585858685878588858985908591859285938594859585968597859885998600860186028603860486058606860786088609861086118612861386148615861686178618861986208621862286238624862586268627862886298630863186328633863486358636863786388639864086418642864386448645864686478648864986508651865286538654865586568657865886598660866186628663866486658666866786688669867086718672867386748675867686778678867986808681868286838684868586868687868886898690869186928693869486958696869786988699870087018702870387048705870687078708870987108711871287138714871587168717871887198720872187228723872487258726

1042110422104231042410425104261042710428104291043010431104321043310434104351043610437104381043910440104411044210443104441044510446104471044810449104501045110452104531045410455104561045710458104591046010461104621046310464104651046610467104681046910470104711047210473104741047510476104771047810479104801048110482104831048410485104861048710488104891049010491104921049310494104951049610497104981049910500105011050210503105041050510506105071050810509105101051110512105131051410515105161051710518105191052010521105221052310524105251052610527105281052910530105311053210533105341053510536105371053810539105401054110542105431054410545105461054710548105491055010551105521055310554105551055610557105581055910560105611056210563105641056510566105671056810569105701057110572105731057410575105761057710578105791058010581105821058310584105851058610587105881058910590105911059210593105941059510596105971059810599106001060110602106031060410605106061060710608106091061010611106121061310614106151061610617106181061910620

1206012061120621206312064120651206612067120681206912070120711207212073120741207512076120771207812079120801208112082120831208412085120861208712088120891209012091120921209312094120951209612097120981209912100121011210212103121041210512106121071210812109121101211112112121131211412115121161211712118121191212012121121221212312124121251212612127121281212912130121311213212133121341213512136121371213812139121401214112142121431214412145121461214712148121491215012151121521215312154121551215612157121581215912160121611216212163121641216512166121671216812169121701217112172121731217412175121761217712178121791218012181121821218312184121851218612187121881218912190121911219212193121941219512196121971219812199122001220112202122031220412205122061220712208122091221012211122121221312214122151221612217122181221912220122211222212223122241222512226122271222812229122301223112232122331223412235122361223712238122391224012241122421224312244122451224612247122481224912250122511225212253122541225512256122571225812259

In [279]:
reformat_data = []
for ed in extra_data:
    reformat_data.append(data_reformat(ed))
    

In [280]:
#make list of dictionaries into a pandas dataframe
feature_df = pd.DataFrame(reformat_data)

In [282]:
#combine total data and feature data
data_total = pd.concat([data,feature_df],axis=1)

In [288]:
data_total

,type,dateCreated,datePublished,reviewBody,url,author_type,author_name,url.1,content_type,movie_name,...,Genre,Director,Writer,Actor,Plot,Language,Country,Poster,Production,Response
0,Review,"Jul 2, 2021","Jul 2, 2021",Consider this film a master class in world-bui...,https://www.nytimes.com/2021/07/01/movies/no-7...,Person,Isabelia Herrera,https://www.rottentomatoes.com/critic/isabelia...,Movie,No.7 Cherry Lane,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False
1,Review,"Jul 1, 2021","Jul 1, 2021",Clichés rain as fast and as furiously as bullets.,https://www.nytimes.com/2021/07/01/movies/the-...,Person,Jeannette Catsoulis,https://www.rottentomatoes.com/critic/jeannett...,Movie,The Tomorrow War,...,"Action, Adventure, Sci-Fi, Thriller",Chris McKay,Zach Dean,"Chris Pratt, Yvonne Strahovski, J.K. Simmons, ...",A family man is drafted to fight in a future w...,English,USA,https://m.media-amazon.com/images/M/MV5BNTI2YT...,"Skydance Media, Phantom Four Films, Lit Entert...",True
2,Review,"Jul 1, 2021","Jul 1, 2021",A sheepish pall steadily creeps over the whole...,https://www.nytimes.com/2021/07/01/movies/long...,Person,Nicolas Rapold,https://www.rottentomatoes.com/critic/nicolas-...,Movie,Long Story Short,...,"Comedy, Romance",Josh Lawson,Josh Lawson,"Rafe Spall, Zahra Newman, Ronny Chieng",Teddy wakes up the morning after his wedding t...,N/A,Australia,https://m.media-amazon.com/images/M/MV5BMjM1ZT...,"Spectrum Films, Create NSW, See Pictures, Scre...",True
3,Review,"Jul 1, 2021","Jul 1, 2021",Another infant adventure that hits the same no...,https://www.nytimes.com/2021/07/01/movies/the-...,Person,Maya Phillips,https://www.rottentomatoes.com/critic/maya-phi...,Movie,The Boss Baby: Family Business,...,"Animation, Adventure, Comedy",Tom McGrath,"Michael McCullers, Tom McGrath, Marla Frazee","Alec Baldwin, James Marsden, Amy Sedaris",The Templeton brothers have become adults and ...,English,United States,https://m.media-amazon.com/images/M/MV5BYWY1N2...,DreamWorks Animation,True
4,Review,"Jul 1, 2021","Jul 1, 2021",Along with some very funny footage of a master...,https://www.nytimes.com/2021/07/01/movies/the-...,Person,Jason Zinoman,https://www.rottentomatoes.com/critic/jason-zi...,Movie,The One and Only Dick Gregory,...,Documentary,Andre Gaines,Andre Gaines,"Chris Rock, Kevin Hart, Lena Waithe",Chronicles the incredible life and times of le...,N/A,United States,https://m.media-amazon.com/images/M/MV5BMDM4NT...,N/A,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227,Review,"Jan 1, 2000","Jan 1, 2000",Dream for an Insomniac is really a self-consci...,http://www.nytimes.com/library/film/061998drea...,Person,Stephen Holden,https://www.rottentomatoes.com/critic/stephen-...,Movie,Dream for an Insomniac,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False
13228,Review,"Jan 1, 2000","Jan 1, 2000",As the movie pops in and out of distinct scena...,https://archive.nytimes.com/www.nytimes.com/li...,Person,Elvis Mitchell,https://www.rottentomatoes.com/critic/elvis-mi...,Movie,The Cell,...,"Crime, Horror, Sci-Fi",Tarsem Singh,Mark Protosevich,"Jennifer Lopez, Vince Vaughn, Vincent D'Onofrio","An F.B.I. Agent persuades a social worker, who...",English,"United States, Germany",https://m.media-amazon.com/images/M/MV5BMDY1Nm...,"Namibia Film Commission, Katira Productions Gm...",True
13229,Review,"Jan 1, 2000","Jan 1, 2000","It's a meal you may feel you've eaten before, ...",NaN,Person,A.O. Scott,https://www.rottentomatoes.com/critic/ao-scott,Movie,What's Cooking?,...,"Comedy, Drama, Romance",Gurinder Chadha,"Gurinder Chadha, Paul Mayeda Berges","Joan Chen, Julianna Margulies, Mercedes Ruehl",Four families in LA of different ethnicity (La...,"English, Spanish, Vietnamese, Yiddish","United Kingdom, United States",https://m.media-amazon.com/images/M/MV5BMTgxMD...,"Hope and Gravy Inc., Stagescreen Productions, ...",True
13230,Review,"Jan 1, 2000","Jan 1, 2000",Although Dance With Me disappoints on almost e...,NaN,Pe

In [293]:
#filter only english movies
data_total_english = data_total[data_total['Language'].str.contains('English')]

In [309]:
#filter only studio movies 
data_total_english_studio = data_total_english[data_total_english['Production'] != 'N/A']

In [317]:
data_total_english_studio = data_total_english_studio.reset_index(drop = True)

In [323]:
data_total_english_studio.to_csv("data_total_english_studio.csv")

In [2]:
data_total_english_studio = pd.read_csv("data_total_english_studio.csv")

In [11]:
#download all posters
for n in range(len(data_total_english_studio)):
    if not data_total_english_studio['Poster'][n] == "N/A":
        try:
            wget.download(data_total_english_studio['Poster'][n],out = "poster/{}.jpg".format(data_total_english_studio['movie_name'][n].replace("?","").replace("/","")))
        except:
            pass

100% [..............................................................................] 32395 / 32395

In [12]:
#remove empty poster images
for file in os.listdir("poster"):
    if(not ".jpg" in file):
        os.remove("poster/{}".format(file))

In [15]:
#remove any rows where there is a N/A
for col in ['Rated','Runtime','Genre','Director','Writer','Actor','Plot','Country']:
    data_total_english_studio = data_total_english_studio[data_total_english_studio[col] != "N/A"]

In [25]:
#check which movies do not have a poster
poster = []
posters = [x.replace(".jpg","") for x in os.listdir("poster")]
for m in data_total_english_studio['movie_name']:
    if(not m.replace("?","").replace("/","") in posters):
        poster.append(False)
    else:
        poster.append(True)
data_total_english_studio['poster'] = poster

In [26]:
#only keep movies with poster
data_total_english_studio = data_total_english_studio[data_total_english_studio['poster'] == True]

,Unnamed: 0,type,dateCreated,datePublished,reviewBody,url,author_type,author_name,url.1,content_type,...,Director,Writer,Actor,Plot,Language,Country,Poster,Production,Response,poster
0,0,Review,"Jul 1, 2021","Jul 1, 2021",Clichés rain as fast and as furiously as bullets.,https://www.nytimes.com/2021/07/01/movies/the-...,Person,Jeannette Catsoulis,https://www.rottentomatoes.com/critic/jeannett...,Movie,...,Chris McKay,Zach Dean,"Chris Pratt, Yvonne Strahovski, J.K. Simmons, ...",A family man is drafted to fight in a future w...,English,USA,https://m.media-amazon.com/images/M/MV5BNTI2YT...,"Skydance Media, Phantom Four Films, Lit Entert...",True,True
2,2,Review,"Jul 1, 2021","Jul 1, 2021","In his feature directing debut, S.K. Dale orch...",https://www.nytimes.com/2021/07/01/movies/till...,Person,Beatrice Loayza,https://www.rottentomatoes.com/critic/beatrice...,Movie,...,S.K. Dale,Jason Carvey,"Megan Fox, Lili Rich, Callan Mulvey, Eoin Macken",A woman is left handcuffed to her dead husband...,English,USA,https://m.media-amazon.com/images/M/MV5BYjU0OT...,Millennium Media,True,True
3,3,Review,"Jul 1, 2021","Jul 1, 2021","Words like ""colonialism"" and ""the American dre...",https://www.nytimes.com/2021/07/01/movies/the-...,Person,Lena Wilson,https://www.rottentomatoes.com/critic/lena-wilson,Movie,...,Everardo Gout,"James DeMonaco, James DeMonaco (characters)","Ana de la Reguera, Will Patton, Josh Lucas, Ca...",All the rules are broken as a sect of lawless ...,English,USA,https://m.media-amazon.com/images/M/MV5BMGE5Nj...,"Perfect World Pictures, Platinum Dunes, Blumho...",True,True
4,4,Review,"Jul 1, 2021","Jul 1, 2021",For the most part a tight and twisty against-t...,https://www.nytimes.com/2021/07/01/movies/no-s...,Person,A.O. Scott,https://www.rottentomatoes.com/critic/ao-scott,Movie,...,Steven Soderbergh,Ed Solomon,"Amy Seimetz, Brendan Fraser, Noah Jupe, Matt D...",A group of criminals are brought together unde...,English,USA,https://m.media-amazon.com/images/M/MV5BNWI2ZD...,"HBO Max, Warner Max, Warner Bros.",True,True
5,5,Review,"Jun 30, 2021","Jun 30, 2021",The doc feels like science-fiction without the...,https://www.nytimes.com/2021/06/25/movies/fath...,Person,Amy Nicholson,https://www.rottentomatoes.com/critic/amy-nich...,Movie,...,Drew Xanthopoulos,NaN,"Michelle Fournet, Ellen Garland",Fathom is a visual and aural wonder of a docum...,English,United States,https://m.media-amazon.com/images/M/MV5BNjc4Zj...,"Hidden Candy, Impact Partners, Back Allie Prod...",True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3697,3697,Review,"Jan 1, 2000","Jan 1, 2000",NaN,NaN,Person,Stephen Holden,https://www.rottentomatoes.com/critic/stephen-...,Movie,...,Martin Davidson,"Jeffrey Goldenberg, Robert Held, Martin Davidson","Armand Assante, Diane Venora, Joe Grifasi, Tom...",The lead singer of an oldies group reminisces ...,English,USA,https://m.media-amazon.com/images/M/MV5BMTQwNz...,"Steve Tisch Company, Echo Productions",True,True
3698,3698,Review,"Jan 1, 2000","Jan 1, 2000",The direction is proficient,NaN,Person,Elvis Mitchell,https://www.rottentomatoes.com/critic/elvis-mi...,Movie,...,Eric Blakeney,Eric Blakeney,"Liam Neeson, Sandra Bullock, Oliver Platt",An undercover DEA agent brings down the Mafia.,English,United States,https://m.media-amazon.com/images/M/MV5BZWIyYW...,"Fortis Films, Hollywood Pictures",True,True
3699,3699,Review,"Jan 1, 2000","Jan 1, 2000",Gladiator suggests what would happen if someon...,http://www.nytimes.com/packages/html/movies/be...,Person,Elvis Mitchell,https://www.rottentomatoes.com/critic/elvis-mi...,Movie,...,Ridley Scott,"David Franzoni (story), David Franzoni (screen...","Russell Crowe, Joaquin Phoenix, Connie Nielsen...",A former Roman General sets out to exact venge...,English,"USA, UK, Malta, Morocco",https://m.media-amazon.com/images/M/MV5BMDliMm...,"Universal Pictures, Scott Free Productions, Dr...",True,True
3700,3700,Review,"Jan 1, 2000","Jan 1, 2000

In [27]:
#save as csv
data_total_english_studio.to_csv("data_total_english_studio_final.csv")

In [225]:
s = small_total['Actor']

In [226]:
pd.DataFrame(mlb.fit_transform(s),columns = mlb.classes_)

,Abbie Cornish,Abigail Hawk,Acoryé White,Adam Goldberg,Al Di,Al Pacino,Alastair Love,Alfre Woodard,Amy Sedaris,Ana de la Reguera,...,Shiloh Fernandez,Silvano Beltrametti,Suki Waterhouse,Tarana Burke,Terrence Howard,Theo Anthony,Tony Hale,Tyrese Gibson,Wolfgang Puck,Zaron Burnett
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [276]:
data_total['Genre'] = data_total['Genre'].apply(lambda x: x.split(","))
data_total['Director'] = data_total['Director'].apply(lambda x: x.split(","))
data_total['Writer'] = data_total['Writer'].apply(lambda x: x.split(","))
data_total['Actor'] = data_total['Actor'].apply(lambda x: x.split(","))
data_total['Language'] = data_total['Language'].apply(lambda x: x.split(","))
data_total['Country'] = data_total['Country'].apply(lambda x: x.split(","))